In [1]:
import pickle
import math
import time
import pandas as pd
import numpy as np
import multiprocessing


In [2]:
term_dict = {}
dict_index = {}
# 对应商品id号的标题分词
cloth_items = []
# 商品的id号
items_id = []
# 保存的tf次数
tf_times = 0
# 保存的idf次数
idf_times = 0
idf = []
have_idf = 0
have_dict = 0

### 处理分词结果  
对dim_items表进行处理，提取出分词结果信息并进行分割，以item_id作为索引存入字典中。

In [3]:
with open('dim_items_new.txt', 'r') as f:
    lines = f.readlines()
    for line in lines:
        fc = line.split()
        # item_id:480073缺少分词数据，NAN补全
        if len(fc) == 2:
            print(line)
        fenci = fc[2]
        items_id.append(fc[0])
        items = fenci.split(',')
        cloth_items.append(items)
        flag = []
        for item in items:
            if item not in term_dict:
                term_dict[item] = 1
            elif item not in flag:
                term_dict[item] += 1
            flag.append(item)


new_dict = {}
index = 0
for term in term_dict:
    # 将分词结果小于4的数据剔除(可修改，主要目的还是提出个别分词结果缺乏的商品信息，如480073)
    if term_dict[term] > 4:
        new_dict[term] = term_dict[term]
        dict_index[term] = index
        index += 1
term_dict = new_dict

### 保存TF值

In [4]:
def single_tf(index):
        term_vector = [0 for i in range(len(term_dict))]
        for item in cloth_items[index]:
            if item in term_dict:
                index = dict_index[item]
                term_vector[index] += 1
        N = sum(term_vector)
        if N == 0:
            print("序号为{0}的商品标题与其他商品都没有大的相似性".format(index))
        # 处理，不要没有太大相关性的商品？
        # self.items_id[index] = -1
        # term_vector = 0
        else:
            term_vector = [term / N for term in term_vector]
        return term_vector

```
terms_vector = []
i = 1
for cloth in range(len(cloth_items)):
    term_vector = single_tf(cloth)
    # if term_vector != 0:
    terms_vector.append(term_vector)
    # 运行时间太长，以此标记存储时间和运行状态
    if cloth and (cloth + 1) % 5000 == 0 or cloth == (len(cloth_items) - 1):
        # print(terms_vector)
        tf_times += 1
        if tf_times % 10 == 0:
            print("已经保存{0}次,共要保存100次".format(tf_times))
        # pickle解决存储问题
        with open('tf_save/tf_save_'+ str(i) + '.pkl','wb') as f:
            pickle.dump(terms_vector, f)
            terms_vector = []
            i = i + 1
```

### 保存idf值

```
N = len(cloth_items)
terms_vector = []
with open('idf_save.pkl', 'wb') as f:
    for term in term_dict:
        # print(self.term_dict[term])
        idf = math.log((N / (term_dict[term] + 1)) + 1)
        terms_vector.append(idf)
    pickle.dump(terms_vector, f)
    terms_vector = []
```

### 保存tf_idf值

```
tf_idf = {}
items_index = 0
idf = pd.read_pickle("idf_save.pkl")
for i in range(1, tf_times):
    tf = pd.read_pickle('tf_save/tf_save_'+ str(i) +'.pkl')
    for tf_item in range(len(tf)):
        # 计算TF-IDF,存入temp
        temp = [a * b for a, b in zip(idf, tf[tf_item])]
        tf_idf[items_id[items_index]] = temp
        items_index += 1
        # 记录，每500个数据存入一次数据
        if tf_item and (tf_item + 1) % 5000 == 0 or tf_item == (len(tf) - 1):
            with open('tf_idf/tf_idf_save' + str(i) + '.pkl', 'wb') as tf_idf_File:  # 因为系统内存不足，在260次停止运行，总文档大致有90G
                print(tf_idf_File)
                pickle.dump(tf_idf, tf_idf_File)
                tf_idf = {}
```

In [5]:
tf_times

0

In [6]:
def single_id_tf(cloth_id):
    index = items_id.index(cloth_id)
    term_vector = single_tf(index)
    return term_vector

def single_tf_idf(cloth_id):
    idf = pd.read_pickle("idf_save.pkl")
    term_tf = single_id_tf(cloth_id)
    term_tf_idf = [a * b for a, b in zip(idf, term_tf)]
    return term_tf_idf

 # 点积
def dot_product(v1, v2):
    return sum(a * b for a, b in zip(v1, v2))

def magnitude(v1):
    return math.sqrt(dot_product(v1, v1))

# 计算余弦相似度
def similarity(v1, v2):
    return dot_product(v1, v2) / (magnitude(v1) * magnitude(v2) + 0.000000000000001)

# 获取tf_idf
def get_tf_idf(item_id):
    tf_idf = single_tf_idf(item_id)
    return tf_idf

def load_idf():
    idf = pd.read_pickle('idf_save.pkl')
        
def sort():
    print("准备进行排序...")
    b = pd.read_pickle("similarity/sim_16.pkl")
    sim_list = []
    for i in range(len(b)):
        sim = sorted(b[i].items(), key=lambda item: item[1], reverse=True)
        sim = sim[0:2000]
        sim = dict(sim)
        print('这是第{0}个数据...'.format(i+1))
        for k,v in dictionary.items():
            if items_id[i] in dictionary[k]:
                key = k
                print(key)
            for j in list(sim.keys()):
                if j in dictionary[k]:
                    key1 = k
                    if key == key1:
                        del sim[j]
        print(len(sim))
        sim_list.append(sim)
    return sim_list

In [7]:
def cat_items():
    f = open('dim_items_new.txt', mode='r', encoding='utf-8')
    line = f.readline()
    list1 = []
    list2 = []
    dictionary = {}
    while line:
        a = line.split()
        b = a[0:1]  # 商品ID
        c = a[1:2]  # 商品类目ID
        list1.append(b[0])
        list2.append(c[0])
        line = f.readline()
    f.close()
    keys = list2
    values = list1

    for i in range(len(keys)):
        dictionary.setdefault(keys[i], []).append(values[i])
    return dictionary

### 相似度计算

In [8]:
# 相似度计算
def sim(term_id):
 
    simi_dict = {}

    with open("dim_items_new.txt",'r') as f:
        line = f.readline()
        list1 = []
        while line:
            a = line.split(' ')
            # item_id
            b = a[0:1]
            list1.append(int(b[0]))
            line = f.readline()
    f.close()
    list2 = []
    for i in range(len(list1)):
        if (i+1)%5000 ==0:
            list2.append(list1[i])
        i = i + 1
    i = 0
    for j in list2:
        if int(term_id) > j and int(term_id)< 3237244:
            i = i + 1
        elif int(term_id) >= 3237244:
            print('打开最后一个文档...')
            tf_idf = pd.read_pickle('tf_idf/tf_idf_save' + str(i+1) + '.pkl')
            search_tf_idf = tf_idf[term_id]
            break
        else:
            print('打开第{0}个文档...'.format(i+1))
            tf_idf = pd.read_pickle('tf_idf/tf_idf_save' + str(i+1) + '.pkl')
            search_tf_idf = tf_idf[term_id]
            break

    for j in range(100): # 待优化
        tf_idf = pd.read_pickle('tf_idf/tf_idf_save' + str(j+1) + '.pkl')
        print('正在比较第' + str(j+1) + '个文档数据...')
        for m in range(len(tf_idf)):
            try:
                simi_dict[items_id[5000*j+m]] = similarity(search_tf_idf, tf_idf[items_id[5000*j+m]]) # tf_idf[] 返回的是列表
            except:
                print('商品{0}存储失败'.format(items_id[5000*j+m]))
                continue
    return simi_dict

In [9]:
if __name__ == "__main__":
    time1 = time.time()
    print("start")
    with open("similarity/sim_100.pkl","wb") as f:
        pickle.dump(a,f)
    f.close()
    dictionary = cat_items()
    print('dict has been loaded...')
    s = multiprocessing.Pool(3)
    b = s.map(sim,items_id[0:3])
    s.close()
    s.join()
    with open("similarity/sim_100.pkl","wb") as f:
        pickle.dump(b,f)
    time2 = time.time()
    time = time2 - time1
    print('耗时： {:.0f}m {:.0f}s'.format(time // 60, time % 60))


start
dict has been loaded...
打开第1个文档...
打开第1个文档...
打开第1个文档...
正在比较第1个文档数据...
正在比较第1个文档数据...
正在比较第1个文档数据...
正在比较第2个文档数据...
正在比较第2个文档数据...
正在比较第2个文档数据...
正在比较第3个文档数据...
正在比较第3个文档数据...
正在比较第3个文档数据...
正在比较第4个文档数据...
正在比较第4个文档数据...
正在比较第4个文档数据...
正在比较第5个文档数据...
正在比较第5个文档数据...
正在比较第5个文档数据...
正在比较第6个文档数据...
正在比较第6个文档数据...
正在比较第6个文档数据...
正在比较第7个文档数据...
正在比较第7个文档数据...
正在比较第7个文档数据...
正在比较第8个文档数据...
正在比较第8个文档数据...
正在比较第8个文档数据...
正在比较第9个文档数据...
正在比较第9个文档数据...
正在比较第9个文档数据...
正在比较第10个文档数据...
正在比较第10个文档数据...
正在比较第10个文档数据...
正在比较第11个文档数据...
正在比较第11个文档数据...
正在比较第11个文档数据...
正在比较第12个文档数据...
正在比较第12个文档数据...
正在比较第12个文档数据...
正在比较第13个文档数据...
正在比较第13个文档数据...
正在比较第13个文档数据...
正在比较第14个文档数据...
正在比较第14个文档数据...
正在比较第14个文档数据...
正在比较第15个文档数据...
正在比较第15个文档数据...
正在比较第15个文档数据...
正在比较第16个文档数据...
正在比较第16个文档数据...
正在比较第16个文档数据...
正在比较第17个文档数据...
正在比较第17个文档数据...
正在比较第17个文档数据...
正在比较第18个文档数据...
正在比较第18个文档数据...
正在比较第18个文档数据...
正在比较第19个文档数据...
正在比较第19个文档数据...
正在比较第19个文档数据...
正在比较第20个文档数据...
正在比较第20个文档数据...
正在比较第20个文档数据...
正在比较

TypeError: can't pickle _thread.lock objects

In [14]:
import time
dictionary = cat_items()
time1 = time.time()
b = pd.read_pickle("similarity/sim_16.pkl")
sim_list = []
for i in range(len(b)):
    sim = sorted(b[i].items(), key=lambda item: item[1], reverse=True)
    sim = sim[0:2000]
    sim = dict(sim)
    print('这是第{0}个数据...'.format(i+1))
    for k,v in dictionary.items():
        if items_id[i] in dictionary[k]:
            key = k
            print(key)
        for j in list(sim.keys()):
            if j in dictionary[k]:
                key1 = k
                if key == key1:
                    del sim[j]
    print(len(sim))
    sim_list.append(sim)
time2 = time.time()
timees = time2 - time1
print('耗时： {:.0f}m {:.0f}s'.format(timees // 60, timees % 60))

这是第1个数据...
52
331
这是第2个数据...
368
167
这是第3个数据...
505
230
这是第4个数据...
505
1186
这是第5个数据...
337
1224
这是第6个数据...
155
1382
这是第7个数据...
584
1405
这是第8个数据...
42
1363
这是第9个数据...
461
818
这是第10个数据...
228
465
这是第11个数据...
52
1044
这是第12个数据...
33
1428
这是第13个数据...
284
597
这是第14个数据...
236
119
这是第15个数据...
111
91
这是第16个数据...
368
173
耗时： 4m 51s


In [30]:
sorted(sim_list[7].items(), key=lambda item: item[1], reverse=True)

[('2600981', 0.7910043139274849),
 ('1294010', 0.7751827864256983),
 ('840490', 0.7742141390488476),
 ('2576516', 0.7711479787687051),
 ('1973909', 0.7613994262646877),
 ('2256927', 0.7611553907405618),
 ('2693265', 0.7586055582331557),
 ('2581173', 0.7550299877675651),
 ('1558010', 0.7550299877675651),
 ('2319544', 0.7376870428331462),
 ('1837001', 0.7368454463123335),
 ('1383369', 0.7349733852168625),
 ('969886', 0.7349250504477638),
 ('3213065', 0.7339348818430113),
 ('714993', 0.7339348818430113),
 ('1191654', 0.7317549192454564),
 ('906562', 0.7306715111343353),
 ('2656178', 0.7304635584310712),
 ('175132', 0.7304059883855814),
 ('2483131', 0.7277628674130362),
 ('2373687', 0.7268800237032057),
 ('2379932', 0.7262515562902622),
 ('1698994', 0.7259604427996302),
 ('809492', 0.7258662973860445),
 ('232895', 0.723047645222077),
 ('3246646', 0.7208401168589862),
 ('149587', 0.7208401168589862),
 ('668945', 0.719820865720713),
 ('1745223', 0.719440871890838),
 ('2234095', 0.71710273787

In [18]:
items_id[0]

'1'

### 测试...

In [59]:
len(tf_idf)

NameError: name 'tf_idf' is not defined

In [33]:
a = sorted(b[0].items(),key=lambda x:x[1],reverse=True)

In [62]:
time1 = time.time()
with open("tf_idf/tf_idf_save1.pkl",'rb') as f:
    pickle.load(f)
time2 = time.time()
time = time2 - time1
print('耗时： {:.0f}m {:.0f}s'.format(time // 60, time % 60))


time1 = time.time()
f = pd.read_pickle("tf_idf/tf_idf_save1.pkl")
time2 = time.time()
time = time2 - time1
print('耗时： {:.0f}m {:.0f}s'.format(time // 60, time % 60))

AttributeError: 'float' object has no attribute 'time'

In [9]:
import time
dictionary = cat_items()
def test(term_id):
    simi_dict = {}
    with open("dim_items_new.txt",'r') as f:
        line = f.readline()
        list1 = []
        while line:
            a = line.split(' ')
            # item_id
            b = a[0:1]
            list1.append(int(b[0]))
            # list2.append(c[0])
            line = f.readline()
    f.close()
    list2 = []
    for i in range(len(list1)):
        if (i+1)%5000 ==0:
            list2.append(list1[i])
        i = i + 1
    i = 0
    for j in list2:
        if int(term_id) > j and int(term_id)< 3237244:
            i = i + 1
        elif int(term_id) >= 3237244:
            print('打开最后一个文档...')
            tf_idf = pd.read_pickle('tf_idf/tf_idf_save' + str(i+1) + '.pkl')
            search_tf_idf = tf_idf[term_id]
            break
        else:
            print('打开第{0}个文档...'.format(i+1))
            tf_idf = pd.read_pickle('tf_idf/tf_idf_save' + str(i+1) + '.pkl')
            search_tf_idf = tf_idf[term_id]
            break
    for j in range(100): # 待优化  
        time1 = time.time()
        tf_idf = pd.read_pickle('tf_idf/tf_idf_save' + str(j+1) + '.pkl')
        print('正在比较第' + str(j+1) + '个文档数据...')
        for m in range(len(tf_idf)):
            try:
                simi_dict[items_id[5000*j+m]] = similarity(search_tf_idf, tf_idf[items_id[5000*j+m]]) # tf_idf[] 返回的是列表
                # print('储存...')
                # simi.setdefault(term_id, []).append(similarity(search_tf_idf, tf_idf[items_id[5000*j+m]]))
                # simi_dict.setdefault(items_id[5000*j+m])
                pass
            except:
                print('商品{0}存储失败'.format(items_id[5000*j+m]))
                continue
        time2 = time.time()
        timees = time2 - time1
        print('耗时： {:.0f}m {:.0f}s'.format(timees // 60, timees % 60))
    return simi_dict
    
if __name__ == "__main__":
    print("start")
    s = multiprocessing.Pool(8)
    print('please waiting AMOMENT...')
    b = s.map(test,items_id[0:8])
    s.close()
    s.join()
    

start
please waiting AMOMENT...
打开第1个文档...
打开第1个文档...
打开第1个文档...打开第1个文档...

打开第1个文档...打开第1个文档...

打开第1个文档...
打开第1个文档...
正在比较第1个文档数据...
正在比较第1个文档数据...
正在比较第1个文档数据...
正在比较第1个文档数据...
正在比较第1个文档数据...
正在比较第1个文档数据...
正在比较第1个文档数据...
正在比较第1个文档数据...
耗时： 0m 47s
耗时： 0m 47s
耗时： 0m 47s
耗时： 0m 47s
耗时： 0m 47s
耗时： 0m 48s
耗时： 0m 48s
耗时： 0m 49s
正在比较第2个文档数据...
正在比较第2个文档数据...
正在比较第2个文档数据...
正在比较第2个文档数据...
正在比较第2个文档数据...
正在比较第2个文档数据...
正在比较第2个文档数据...
正在比较第2个文档数据...
耗时： 0m 45s
耗时： 0m 45s
耗时： 0m 46s
耗时： 0m 45s
耗时： 0m 46s
耗时： 0m 47s
耗时： 0m 47s
耗时： 0m 46s
正在比较第3个文档数据...
正在比较第3个文档数据...
正在比较第3个文档数据...
正在比较第3个文档数据...
正在比较第3个文档数据...
正在比较第3个文档数据...
正在比较第3个文档数据...
正在比较第3个文档数据...
耗时： 0m 46s
耗时： 0m 47s
耗时： 0m 46s
耗时： 0m 46s
耗时： 0m 47s
耗时： 0m 48s
耗时： 0m 48s
耗时： 0m 49s
正在比较第4个文档数据...
正在比较第4个文档数据...
正在比较第4个文档数据...
正在比较第4个文档数据...
正在比较第4个文档数据...
正在比较第4个文档数据...
正在比较第4个文档数据...
正在比较第4个文档数据...
耗时： 0m 45s
耗时： 0m 47s
耗时： 0m 46s
耗时： 0m 47s
耗时： 0m 46s
耗时： 0m 47s
耗时： 0m 47s
耗时： 0m 47s
正在比较第5个文档数据...
正在比较第5个文档数据...
正在比较第5个文档数据...
正在比

Process ForkPoolWorker-8:
Traceback (most recent call last):
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()


KeyboardInterrupt: 

  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.5/multiprocessing/pool.py", line 119, in worker
    result = (True, func(*args, **kwds))
  File "/usr/lib/python3.5/multiprocessing/pool.py", line 44, in mapstar
    return list(map(*args))
  File "<ipython-input-9-5674e7ebc6dc>", line 38, in test
    print('正在比较第' + str(j+1) + '个文档数据...')
KeyboardInterrupt


耗时： 0m 48s


Process ForkPoolWorker-2:
Traceback (most recent call last):
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.5/multiprocessing/pool.py", line 119, in worker
    result = (True, func(*args, **kwds))
  File "/usr/lib/python3.5/multiprocessing/pool.py", line 44, in mapstar
    return list(map(*args))
  File "<ipython-input-9-5674e7ebc6dc>", line 38, in test
    print('正在比较第' + str(j+1) + '个文档数据...')
KeyboardInterrupt
Process ForkPoolWorker-1:
Traceback (most recent call last):
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.5/multiprocessing/pool.py", line 119, in worker
    result = (True, func(*args, **kwds))
  Fi

耗时： 0m 47s


Process ForkPoolWorker-4:
Traceback (most recent call last):
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.5/multiprocessing/pool.py", line 119, in worker
    result = (True, func(*args, **kwds))
  File "/usr/lib/python3.5/multiprocessing/pool.py", line 44, in mapstar
    return list(map(*args))
  File "<ipython-input-9-5674e7ebc6dc>", line 37, in test
    tf_idf = pd.read_pickle('tf_idf/tf_idf_save' + str(j+1) + '.pkl')
  File "/usr/local/lib/python3.5/dist-packages/pandas/io/pickle.py", line 179, in read_pickle
    return try_read(path)
  File "/usr/local/lib/python3.5/dist-packages/pandas/io/pickle.py", line 166, in try_read
    return read_wrapper(lambda f: pkl.load(f))
  File "/usr/local/lib/python3.5/dist-packages/pandas/io/pickle.py", line 148, in read_wrapper
    return func(f)
  File "/us

正在比较第13个文档数据...
正在比较第13个文档数据...
耗时： 0m 54s
耗时： 0m 55s
正在比较第14个文档数据...
正在比较第14个文档数据...
耗时： 0m 53s
耗时： 0m 52s
正在比较第15个文档数据...
正在比较第15个文档数据...
耗时： 0m 56s
耗时： 0m 53s
正在比较第16个文档数据...
正在比较第16个文档数据...
耗时： 0m 55s
耗时： 0m 55s
正在比较第17个文档数据...
正在比较第17个文档数据...
耗时： 0m 53s
耗时： 0m 52s
正在比较第18个文档数据...
正在比较第18个文档数据...
耗时： 0m 56s
耗时： 0m 56s
正在比较第19个文档数据...
正在比较第19个文档数据...
耗时： 0m 53s
耗时： 0m 54s
正在比较第20个文档数据...
正在比较第20个文档数据...
耗时： 0m 54s
耗时： 0m 58s
正在比较第21个文档数据...
正在比较第21个文档数据...
耗时： 0m 54s
耗时： 0m 53s
正在比较第22个文档数据...
正在比较第22个文档数据...
耗时： 1m 1s
耗时： 1m 2s
正在比较第23个文档数据...
正在比较第23个文档数据...
耗时： 1m 2s
耗时： 1m 4s
正在比较第24个文档数据...
正在比较第24个文档数据...
耗时： 0m 58s
耗时： 1m 1s
正在比较第25个文档数据...
正在比较第25个文档数据...
耗时： 0m 56s
耗时： 0m 57s
正在比较第26个文档数据...
正在比较第26个文档数据...
耗时： 0m 57s
耗时： 0m 58s
正在比较第27个文档数据...
正在比较第27个文档数据...
耗时： 0m 58s
耗时： 0m 56s
正在比较第28个文档数据...
正在比较第28个文档数据...
耗时： 0m 59s
耗时： 0m 57s
正在比较第29个文档数据...
正在比较第29个文档数据...
耗时： 0m 60s
耗时： 0m 59s
正在比较第30个文档数据...
正在比较第30个文档数据...
耗时： 1m 3s
耗时： 1m 0s
正在比较第31个文档数据...
正在比较第31个文档数据...
耗时：

Process ForkPoolWorker-5:
Process ForkPoolWorker-6:
  File "/usr/lib/python3.5/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/usr/lib/python3.5/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
Traceback (most recent call last):
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.5/multiprocessing/queues.py", line 342, in get
    with self._rlock:
  File "/usr/lib/python3.5/multiprocessing/queues.py", line

In [ ]:
items_id[0:20]

In [9]:
term_id = '4826'
dictionary = cat_items()
simi_dict = {}
with open("dim_items_new.txt",'r') as f:
    line = f.readline()
    list1 = []
    while line:
        a = line.split(' ')
        # item_id
        b = a[0:1]
        list1.append(int(b[0]))
        # list2.append(c[0])
        line = f.readline()
f.close()
list2 = []
for i in range(len(list1)):
    if (i+1)%5000 ==0:
        list2.append(list1[i])
    i = i + 1
i = 0
for j in list2:
    if int(term_id) > j and int(term_id)< 3237244:
        i = i + 1
    elif int(term_id) >= 3237244:
        print('打开最后一个文档...')
        tf_idf = pd.read_pickle('tf_idf/tf_idf_save' + str(i+1) + '.pkl')
        search_tf_idf = tf_idf[term_id]
        break
    else:
        print('打开第{0}个文档...'.format(i+1))
        tf_idf = pd.read_pickle('tf_idf/tf_idf_save' + str(i+1) + '.pkl')
        search_tf_idf = tf_idf[term_id]
        break

for j in range(100): # 待优化
    time1 = time.time()
    tf_idf = pd.read_pickle('tf_idf/tf_idf_save' + str(j+1) + '.pkl')
    print('正在比较第' + str(j+1) + '个文档数据...')
    for m in range(len(tf_idf)):
        try:
            simi_dict[items_id[5000*j+m]] = similarity(search_tf_idf, tf_idf[items_id[5000*j+m]]) # tf_idf[] 返回的是列表
        except:
            print('商品{0}存储失败'.format(items_id[5000*j+m]))
            continue
    time2 = time.time()
    timees = time2 - time1
    print('耗时： {:.0f}m {:.0f}s'.format(timees // 60, timees % 60))

打开第1个文档...
正在比较第1个文档数据...
耗时： 0m 41s
正在比较第2个文档数据...
耗时： 0m 42s
正在比较第3个文档数据...
耗时： 0m 40s
正在比较第4个文档数据...
耗时： 0m 39s
正在比较第5个文档数据...
耗时： 0m 40s
正在比较第6个文档数据...
耗时： 0m 40s
正在比较第7个文档数据...
商品207647存储失败
耗时： 0m 40s


KeyboardInterrupt: 

In [126]:
b = pd.read_pickle("similarity/sim_10.pkl")

In [29]:
for i in range(len(b)):
    for k,v in dictionary.items():
        for j in b[i].keys():
            if j in dictionary[k]:
                key1 = k


52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
5

KeyboardInterrupt: 

In [51]:
b = pd.read_pickle("similarity/sim_10.pkl")
a = sorted(b[0].items(), key=lambda item: item[1], reverse=True)
a = a[0:5000]
a = dict(a)
a.keys()

dict_keys(['2904989', '3052663', '1662837', '1901626', '2401541', '2476777', '2546820', '1734012', '1141811', '2198409', '3151187', '2128222', '1865136', '2905388', '2092259', '546919', '2943379', '1577147', '2577294', '2861494', '2613939', '2772472', '1700127', '1153258', '2624242', '566911', '611451', '2193091', '804900', '890044', '988201', '446118', '1461804', '1002586', '1441351', '2779579', '886478', '890633', '1316056', '2424498', '2577380', '2816854', '1654631', '2361549', '2987332', '1938205', '518031', '2714836', '2823720', '2167111', '1900333', '844266', '32835', '3082410', '1197220', '336055', '2350713', '1827810', '1645166', '1883806', '1556826', '56244', '2128452', '1140698', '2148278', '1854161', '1299107', '1969354', '3058186', '2858901', '1741729', '743710', '1562856', '1562483', '2531345', '1248851', '2257358', '257188', '3073900', '1181671', '1340660', '2571725', '3207084', '1708610', '342615', '3089954', '2781209', '2841585', '2438750', '1026630', '1951477', '239081

In [87]:
items_id[7]

'43'

In [126]:
list(f.keys())

['43548',
 '43219',
 '60137',
 '48306',
 '44884',
 '48135',
 '56576',
 '57847',
 '64447',
 '41539',
 '46727',
 '36702',
 '58907',
 '52162',
 '39193',
 '63201',
 '63781',
 '48071',
 '52722',
 '51587',
 '43553',
 '34875',
 '64998',
 '50449',
 '42965',
 '58590',
 '47571',
 '37429',
 '38673',
 '55802',
 '56126',
 '44389',
 '62578',
 '62421',
 '41200',
 '55203',
 '35423',
 '47147',
 '34732',
 '49022',
 '61871',
 '47356',
 '59244',
 '62250',
 '48361',
 '50656',
 '62511',
 '63480',
 '40670',
 '65131',
 '57828',
 '56117',
 '38323',
 '60855',
 '64269',
 '60760',
 '38173',
 '62858',
 '60243',
 '56244',
 '39584',
 '40702',
 '36908',
 '52598',
 '40526',
 '49194',
 '39875',
 '62298',
 '47833',
 '59868',
 '39666',
 '35797',
 '37329',
 '35147',
 '39746',
 '43222',
 '40095',
 '50761',
 '49433',
 '47321',
 '33022',
 '56768',
 '33020',
 '35805',
 '37054',
 '44903',
 '60720',
 '34205',
 '35972',
 '63961',
 '48508',
 '41242',
 '58884',
 '36355',
 '34882',
 '34682',
 '54292',
 '59136',
 '46101',
 '38827',


### KMeans聚类尝试

In [169]:
km = pd.read_pickle("train.pkl")

TypeError: 'KMeans' object does not support indexing

In [167]:
import time
time1 = time.time()
kmeans_dic = {}
for j in range(100): # 待优化
    tf_idf = pd.read_pickle('tf_idf/tf_idf_save' + str(j+1) + '.pkl')
    print('正在比较第' + str(j+1) + '个文档数据...')
    for m in range(len(tf_idf)):
        try:
            x = np.array(tf_idf[items_id[5000*j+m]])
            y_pre = km.predict([x])
            kmeans_dic.setdefault(y_pre[0], []).append(items_id[5000*j+m])
        except:
            print('商品{0}存储失败'.format(items_id[5000*j+m]))
            continue
time2 = time.time()
timees = time2 - time1
print('耗时： {:.0f}m {:.0f}s'.format(timees // 60, timees % 60))

正在比较第1个文档数据...
商品1存储失败
商品3存储失败
商品8存储失败
商品12存储失败
商品23存储失败
商品29存储失败
商品31存储失败
商品43存储失败
商品46存储失败
商品49存储失败
商品51存储失败
商品58存储失败
商品59存储失败
商品62存储失败
商品68存储失败
商品75存储失败
商品87存储失败
商品100存储失败
商品102存储失败
商品103存储失败
商品109存储失败
商品119存储失败
商品128存储失败
商品147存储失败
商品154存储失败
商品155存储失败
商品160存储失败
商品179存储失败
商品181存储失败
商品186存储失败
商品201存储失败
商品207存储失败
商品212存储失败
商品217存储失败
商品218存储失败
商品223存储失败
商品230存储失败
商品237存储失败
商品248存储失败
商品250存储失败
商品253存储失败
商品257存储失败
商品264存储失败
商品277存储失败
商品291存储失败
商品305存储失败
商品307存储失败
商品341存储失败
商品343存储失败
商品352存储失败
商品360存储失败
商品365存储失败
商品370存储失败
商品374存储失败
商品388存储失败
商品392存储失败
商品393存储失败
商品400存储失败
商品413存储失败
商品414存储失败
商品420存储失败
商品421存储失败
商品424存储失败
商品432存储失败
商品433存储失败
商品443存储失败
商品444存储失败
商品450存储失败
商品465存储失败
商品466存储失败
商品474存储失败
商品480存储失败
商品486存储失败
商品489存储失败
商品494存储失败
商品495存储失败
商品499存储失败
商品504存储失败
商品508存储失败
商品513存储失败
商品521存储失败
商品527存储失败
商品531存储失败
商品544存储失败
商品559存储失败
商品568存储失败
商品569存储失败
商品575存储失败
商品597存储失败
商品605存储失败
商品606存储失败
商品619存储失败
商品622存储失败
商品632存储失败
商品638存储失败
商品640存储失败
商品642存储失败
商品645存储失败
商品652存储失败
商品657存储失败
商品660

KeyboardInterrupt: 

{'1': 0.99}